## Tensorflow 2.0 speed measurement

In [3]:
import sys
import os
import numpy as np
import pickle
import tensorflow as tf
from sklearn.model_selection import train_test_split
import time
import timeit

project_root = os.path.join(os.getcwd(), '..', '..')
sys.path.append(project_root)

from python.data_processing import data_normalization, data_preprocessing
from python.predictor.models.rnn_cells import LSTMPeepholeCell, ResidualLSTMCell

training_shuffled = pickle.load(open('../../python/data/trainingShuffled.pkl', 'rb'))
train_data = {}
test_data = {}
train_data["Inputs"], test_data["Inputs"], train_data["Outputs"], test_data["Outputs"] = train_test_split(training_shuffled["Inputs"], training_shuffled["Outputs"], test_size=0.1, random_state=42)


In [4]:
print(training_shuffled["Inputs"].shape)
print(training_shuffled["Outputs"].shape)
train_data["Inputs"].shape

normalized_train_data, min, max = data_normalization.minmax_scale_data(train_data)
normalized_test_data, _, _ = data_normalization.minmax_scale_data(test_data, min, max)
norm_in, norm_out = normalized_train_data["Inputs"], normalized_train_data["Outputs"]
preprocessed_train_data = data_preprocessing.lag_features(X=norm_in, y=norm_out, outputs_lag=3)
preprocessed_test_data = data_preprocessing.lag_features(X=norm_in, y=norm_out, outputs_lag=3)

input_shape = preprocessed_train_data["Inputs"].shape[1:]

(450, 100, 2)
(450, 100, 6)


## 1. Measure dependence of training time on networks depth

In [5]:
max_layers = [1,2,4]
units = 20

models = []
for layers in max_layers:
    inputs = tf.keras.Input(shape=input_shape)
    x = inputs
    for i in range(layers):
        x = tf.keras.layers.LSTM(units, return_sequences=True)(x)
    outputs = tf.keras.layers.Dense(6)(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="vanila_lstm_{}_layers".format(layers))
    models.append(model)

    inputs = tf.keras.Input(shape=input_shape)
    x = inputs
    for i in range(layers):
        x = tf.keras.layers.GRU(units, return_sequences=True)(x)
    outputs = tf.keras.layers.Dense(6)(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="gru_{}_layers".format(layers))
    models.append(model)

    inputs = tf.keras.Input(shape=input_shape)
    x = inputs
    for i in range(layers):
        x = tf.keras.layers.RNN(LSTMPeepholeCell(units), return_sequences=True)(x)
    outputs = tf.keras.layers.Dense(6)(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="peephole_lstm_{}_layers".format(layers))
    models.append(model)

    inputs = tf.keras.Input(shape=input_shape)
    x = inputs
    for i in range(layers):
        x = tf.keras.layers.RNN(ResidualLSTMCell(units), return_sequences=True)(x)
    outputs = tf.keras.layers.Dense(6)(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="residual_lstm_{}_layers".format(layers))
    models.append(model)

In [7]:

val_data = (preprocessed_test_data["Inputs"], preprocessed_test_data["Outputs"])
epochs = 10
batch_size = 32
n_measurements = 4

time_measurements = {}
validation_rmses = {}
for model in models:
    print(model.name)
    time_measurements[model.name] = []
    validation_rmses[model.name] = []
    for n in range(n_measurements):
        model.compile(optimizer='adam',
                    loss=tf.keras.losses.MeanSquaredError(),
                    metrics=[tf.keras.metrics.RootMeanSquaredError()])
        start = time.time()
        result = model.fit(preprocessed_train_data["Inputs"], preprocessed_train_data["Outputs"], validation_data=val_data, epochs=epochs, batch_size=batch_size, verbose=0)
        end = time.time()
        fit_time = end - start
        time_measurements[model.name].append(fit_time)
        validation_rmses[model.name].append(result.history["val_loss"][-1])
    time_measurements[model.name] = np.array(time_measurements[model.name])
    validation_rmses[model.name] = np.array(validation_rmses[model.name])

experiment = {}
experiment["Name"] = "Tensorflow training time"
experiment["Description"] = "Measurements of training time for different LSTM cells using Tensorflow"   
experiment["Parameters"] = {}
experiment["Parameters"]["Epochs"] = epochs
experiment["Parameters"]["Batch size"] = batch_size
experiment["Parameters"]["Number of measurements"] = n_measurements
experiment["Parameters"]["Units"] = units
experiment["Time Measurements"] = time_measurements
experiment["Validation RMSE"] = validation_rmses

vanila_lstm_1_layers
gru_1_layers
peephole_lstm_1_layers
residual_lstm_1_layers
vanila_lstm_2_layers
gru_2_layers
peephole_lstm_2_layers
residual_lstm_2_layers
vanila_lstm_4_layers
gru_4_layers
peephole_lstm_4_layers
residual_lstm_4_layers


Save the depth results

In [8]:
with open("data/tensorflow_depth_training_time.pkl", "wb") as f:
    pickle.dump(experiment, f)

## 2. Measure dependence of training time on networks width (units)

In [3]:
layers = 1
units_list = [10, 20, 50, 100, 200, 400]

models = []
for units in units_list:
    inputs = tf.keras.Input(shape=input_shape)
    x = tf.keras.layers.LSTM(units, return_sequences=True)(inputs)
    outputs = tf.keras.layers.Dense(6)(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="vanila_lstm_{}_units".format(units))
    models.append(model)

    inputs = tf.keras.Input(shape=input_shape)
    x = tf.keras.layers.GRU(units, return_sequences=True)(inputs)
    outputs = tf.keras.layers.Dense(6)(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="gru_{}_units".format(units))
    models.append(model)

    inputs = tf.keras.Input(shape=input_shape)
    x = tf.keras.layers.RNN(LSTMPeepholeCell(units), return_sequences=True)(inputs)
    outputs = tf.keras.layers.Dense(6)(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="peephole_lstm_{}_units".format(units))
    models.append(model)

    inputs = tf.keras.Input(shape=input_shape)
    x = tf.keras.layers.RNN(ResidualLSTMCell(units), return_sequences=True)(inputs)
    outputs = tf.keras.layers.Dense(6)(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="residual_lstm_{}_units".format(units))
    models.append(model)

In [4]:
val_data = (preprocessed_test_data["Inputs"], preprocessed_test_data["Outputs"])
epochs = 10
batch_size = 32
n_measurements = 4

time_measurements = {}
validation_rmses = {}
for model in models:
    print(model.name)
    time_measurements[model.name] = []
    validation_rmses[model.name] = []
    for n in range(n_measurements):
        model.compile(optimizer='adam',
                    loss=tf.keras.losses.MeanSquaredError(),
                    metrics=[tf.keras.metrics.RootMeanSquaredError()])
        start = time.time()
        result = model.fit(preprocessed_train_data["Inputs"], preprocessed_train_data["Outputs"], validation_data=val_data, epochs=epochs, batch_size=batch_size, verbose=0)
        end = time.time()
        fit_time = end - start
        time_measurements[model.name].append(fit_time)
        validation_rmses[model.name].append(result.history["val_loss"][-1])
        # TODO: save last validation RMSE as the measured time
    time_measurements[model.name] = np.array(time_measurements[model.name])
    validation_rmses[model.name] = np.array(validation_rmses[model.name])

experiment = {}
experiment["Name"] = "Tensorflow training time"
experiment["Description"] = "Measurements of training time for different LSTM cells using Tensorflow"   
experiment["Parameters"] = {}
experiment["Parameters"]["Epochs"] = epochs
experiment["Parameters"]["Batch size"] = batch_size
experiment["Parameters"]["Number of measurements"] = n_measurements
experiment["Parameters"]["Units"] = units
experiment["Time Measurements"] = time_measurements
experiment["Validation RMSE"] = validation_rmses

vanila_lstm_10_units
gru_10_units
peephole_lstm_10_units
residual_lstm_10_units
vanila_lstm_20_units
gru_20_units
peephole_lstm_20_units
residual_lstm_20_units
vanila_lstm_50_units
gru_50_units
peephole_lstm_50_units
residual_lstm_50_units
vanila_lstm_100_units
gru_100_units
peephole_lstm_100_units
residual_lstm_100_units
vanila_lstm_200_units
gru_200_units
peephole_lstm_200_units
residual_lstm_200_units
vanila_lstm_400_units
gru_400_units
peephole_lstm_400_units
residual_lstm_400_units


Save the width results

In [5]:
with open("data/tensorflow_width_training_time.pkl", "wb") as f:
    pickle.dump(experiment, f)